In [1]:
from glob import glob
from os.path import join, basename
import pickle
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
model = VGG16("vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [3]:
truncated_model = Model(inputs=model.input, outputs=model.get_layer('block5_pool').output)

In [4]:
def process_image(img_path= 'cat.jpg'):
    img = image.load_img(img_path, target_size=(224, 224))
    x_img = image.img_to_array(img)
    x_img = np.expand_dims(x_img, axis=0)
    x_img = preprocess_input(x_img)
    return x_img

In [5]:
conv_features = truncated_model.predict(process_image()).flatten()

In [6]:
conv_features, conv_features.shape

(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), (25088,))

In [7]:
final_dict = {}
all_img_paths = glob(join('flickr30k_images/flickr30k_images', "**.jpg"))

with open("image_embeddings.txt", "w") as o_f:
    for x in all_img_paths:
        result = truncated_model.predict(process_image(x)).flatten()
        print(result, result.shape)
        final_dict[basename(x)] = result
        break

[ 0.        0.        0.       ...  0.       23.054588  0.      ] (25088,)


In [8]:
with open('image_embeddings.pickle', 'wb') as file_handle:
    pickle.dump(final_dict, file_handle, protocol=pickle.HIGHEST_PROTOCOL)

### Not required

In [9]:
with open('image_embeddings.pickle', 'rb') as file_handle:
    b = pickle.load(file_handle)
    print(b)

{'2609797461.jpg': array([ 0.      ,  0.      ,  0.      , ...,  0.      , 23.054588,
        0.      ], dtype=float32)}
